**Load the required libraries and data**

In [1]:
# Libraries
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
import seaborn as sns 

In [2]:
# Data  
appointment_df = pd.read_csv("../data/Appointments List.csv", 
                             encoding='UTF-16', delimiter="\t")

/tmp/ipykernel_121724/1929627687.py:2: DtypeWarning: Columns (0,3,14) have mixed types. Specify dtype option on import or set low_memory=False.
  appointment_df = pd.read_csv("../data/Appointments List.csv",


# Explanatory Data Analysis
## Data Preparation

In [3]:
appointment_df.head()

,Appointment UID,Appt Status Description,Date Of Service,Chartnumber,NurseName,FirstTreatmentApptInd,Appt Type Group,Appt Type,Created By,Provider,Facility City,Room,Column Heading,#,Count of tblAppointments,Patient FID
0,Grand Total,Total,Total,Total,Total,Total,Total,Total,Total,Total,Total,Total,Total,Total,"55,975",*
1,1,Made,01/01/00,NaN,NaN,N,Default,NaN,system,",",NaN,Other,NaN,1,1,1
2,109,Seen,01/22/18,1,NaN,N,Intake,TMS INTAKE DR. BRENNER,EMILY,"BRENNER,DANIEL A",CAMBRIDGE,Other,DR. BRENNER,2,1,"5,283"
3,239,Seen,01/22/18,2,NaN,N,MH Infusion,KETAMINE MH -INFUSION ONLY,EMILY,"BRENNER,DANIEL A",CAMBRIDGE,Other,2C,3,1,"5,286"
4,240,Made,01/22/18,9,NaN,N,Intake,TMS INTAKE DR. BRENNER,EMILY,"BRENNER,DANIEL A",CAMBRIDGE,Other,DR. BRENNER,4,1,"5,351"


In [4]:
appointment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55976 entries, 0 to 55975
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Appointment UID           55976 non-null  object
 1   Appt Status Description   55976 non-null  object
 2   Date Of Service           37250 non-null  object
 3   Chartnumber               55975 non-null  object
 4   NurseName                 19321 non-null  object
 5   FirstTreatmentApptInd     54678 non-null  object
 6   Appt Type Group           54678 non-null  object
 7   Appt Type                 54677 non-null  object
 8   Created By                55976 non-null  object
 9   Provider                  55976 non-null  object
 10  Facility City             51738 non-null  object
 11  Room                      55976 non-null  object
 12  Column Heading            54697 non-null  object
 13  #                         55976 non-null  object
 14  Count of tblAppointmen

In [5]:
# Drop the first two rows 
appointment_df.drop(index=[0, 1], inplace=True)

In [6]:
appointment_df["Chartnumber"] = pd.to_numeric(appointment_df["Chartnumber"], errors="coerce")
appointment_df["Date"] = pd.to_datetime(appointment_df["Date Of Service"], errors="coerce")

/tmp/ipykernel_121724/2432676088.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  appointment_df["Date"] = pd.to_datetime(appointment_df["Date Of Service"], errors="coerce")


In [7]:
# Replace "Made" with "Seen"
appointment_df["Appt Status Description"].replace(to_replace="Made", value="Seen", inplace=True)

In [8]:
appointment_df["Facility City"].isna().sum()

4237

In [9]:
appointment_df["Facility City"] = appointment_df["Facility City"].ffill().backfill()

/tmp/ipykernel_121724/1807988571.py:1: FutureWarning: DataFrame.backfill/Series.backfill is deprecated. Use DataFrame.bfill/Series.bfill instead
  appointment_df["Facility City"] = appointment_df["Facility City"].ffill().backfill()


In [10]:
# Confirm if there are still null values
appointment_df["Facility City"].isna().sum()

0

In [11]:
# Select the successfull appointments(visits)
seen_df = appointment_df[appointment_df["Appt Status Description"]=="Seen"]

# subset the data set 
subset_columns = ["Date", "Chartnumber", "Appt Type Group", "Facility City"]
seen_df = seen_df[subset_columns]

# Drop any record with null values 
seen_df.dropna(how="any", inplace=True)

In [12]:
# Calculate the percentage of patients who start with intake 
count = 0
chart_numbers = seen_df["Chartnumber"].unique()

for chart_no in chart_numbers:
    # Filter the data set 
    df_by_chart_no = seen_df[seen_df["Chartnumber"]==chart_no].sort_values(by="Date")
    df_by_chart_no.reset_index(drop=True, inplace=True)
    
    if df_by_chart_no["Appt Type Group"][0]=="Intake":
        count+=1
    else:
        pass
    
# Patients enrolled with an intake    
count/seen_df["Chartnumber"].nunique()

0.8556767158434894

approximately 86% of the patients enrolled with an intake instead of treatment or follow up

Features/Variables to include in the post intake dataset 

* Chart number 
* First Service after enrollment
* First Service after intake
* Days after intake
* Days between MH Infusion and Intake 
* Number of visits
* Number of MH infusions

* Number of MH infusions outside cambridge 
* Number of visits outside cambridge

In [13]:
first_service_after_enrollment = []
first_service_after_intake = []
days_after_intake = []
days_btn_mhinfusion_intake = []
visit_count = []
mhInfusion_count = []

# Additional Modifications
infusion_count_outside_cambridge = []
visit_count_outside_cambridge = []
intake_facility_city = [] # not yet implemented



for chart_no in chart_numbers:
    # Filter the data set by chart number
    df_by_chart_no = seen_df[seen_df["Chartnumber"]==chart_no].sort_values(by="Date")
    df_by_chart_no.reset_index(drop=True, inplace=True)
    
    # Get the required data 
    first_service_after_enrollment.append(df_by_chart_no["Appt Type Group"][0])
    visit_count.append(df_by_chart_no.shape[0])
    mhInfusion_count.append(df_by_chart_no["Appt Type Group"].to_list().count('MH Infusion'))
    
    # Get the additional data 
    ## Filter out Cambridge visits from the data
    outside_cambridge_df = df_by_chart_no[df_by_chart_no["Facility City"]!="CAMBRIDGE"]
    visit_count_outside_cambridge.append(outside_cambridge_df.shape[0])
    
    ## Count the number of infusions outside cambridge
    infusion_count_outside_cambridge.append(outside_cambridge_df["Appt Type Group"].tolist().count('MH Infusion'))
    
    if df_by_chart_no["Appt Type Group"].tolist()[0] == "Intake":
        # Check if there exists multiple services
        if df_by_chart_no.shape[0] > 1:
            first_service_after_intake.append(df_by_chart_no["Appt Type Group"][1])

            # Calculate days after intake
            days_1 = (df_by_chart_no["Date"][1] - df_by_chart_no["Date"][0]).days
            days_after_intake.append(days_1)

            # Calculate days between intake and MH infusion
            try:
                mhInfusion_index = df_by_chart_no["Appt Type Group"].tolist().index('MH Infusion')
                days_2 = (df_by_chart_no["Date"][mhInfusion_index] - df_by_chart_no["Date"][0]).days
                days_btn_mhinfusion_intake.append(days_2)
            except ValueError: 
                days_btn_mhinfusion_intake.append(np.nan)        
            
        else:
            first_service_after_intake.append(np.nan)
            days_after_intake.append(np.nan)
            days_btn_mhinfusion_intake.append(np.nan)
    
#     # The below line checks if there is another service before intake
#     elif "Intake" in df_by_chart_no["Appt Type Group"].tolist():
#         print(chart_no)
#         # Note the intake index
#         intake_index = df_by_chart_no["Appt Type Group"].tolist().index("Intake")
#         print(intake_index)
        
#         # Get the required values 
#         print(df_by_chart_no["Appt Type Group"].tolist()[intake_index + 1])
#         first_service_after_intake.append(df_by_chart_no["Appt Type Group"].tolist()[intake_index + 1])
        
    else: 
        if "Intake" in df_by_chart_no["Appt Type Group"].tolist():
            print(chart_no)
            
            # Note the intake index
            intake_index = df_by_chart_no["Appt Type Group"].tolist().index("Intake")
            print("Intake index", intake_index)
            print(df_by_chart_no["Appt Type Group"].tolist())
            
            # Get the required values
            print(df_by_chart_no)
            first_service_after_intake.append(df_by_chart_no["Appt Type Group"].tolist()[intake_index + 1])
            
        else:          
        
            first_service_after_intake.append(np.nan)
            days_after_intake.append(np.nan)
            days_btn_mhinfusion_intake.append(np.nan)

118
Intake index 1
['MH Infusion', 'Intake', 'MH Infusion', 'MH Infusion', 'Follow-Up', 'MH Infusion', 'MH Infusion']
        Date  Chartnumber Appt Type Group Facility City
0 2018-01-26          118     MH Infusion     CAMBRIDGE
1 2018-01-26          118          Intake     CAMBRIDGE
2 2018-02-05          118     MH Infusion     CAMBRIDGE
3 2018-02-15          118     MH Infusion     CAMBRIDGE
4 2020-04-21          118       Follow-Up     CAMBRIDGE
5 2020-05-08          118     MH Infusion     CAMBRIDGE
6 2020-06-09          118     MH Infusion     CAMBRIDGE
24
Intake index 13
['TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'Intake', 'TMS', 'MH Infusion', 'TMS', 'TMS', 'TMS', 'MH Infusion', 'MH Infusion', 'TMS', 'TMS', 'MH Infusion', 'MH Infusion', 'TMS', 'TMS', 'MH Infusion', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS', 'TMS',

IndexError: list index out of range

In [ ]:
stop here

In [ ]:
seen_df[(seen_df["Facility City"]!="CAMBRIDGE")&(seen_df["Chartnumber"]==1909)]

In [19]:
seen_df[seen_df["Chartnumber"]==276]

,Date,Chartnumber,Appt Type Group,Facility City
3899,2018-10-15,276,Follow-Up,CAMBRIDGE
40170,2024-07-03,276,Intake,CAMBRIDGE
40449,2023-07-21,276,MH Infusion,AMHERST
40558,2023-07-21,276,MH Infusion,CAMBRIDGE


In [18]:
d1 = seen_df[seen_df["Chartnumber"]==276].sort_values(by="Date", ascending=True)
d1.reset_index(drop=True, inplace=True)
d1

,Date,Chartnumber,Appt Type Group,Facility City
0,2018-10-15,276,Follow-Up,CAMBRIDGE
1,2023-07-21,276,MH Infusion,AMHERST
2,2023-07-21,276,MH Infusion,CAMBRIDGE
3,2024-07-03,276,Intake,CAMBRIDGE


In [ ]:
seen_df[seen_df["Chartnumber"]==276].reset_index()["Appt Type Group"].tolist()

In [ ]:
stop here 

In [ ]:
# Create a dataframe  
post_intake_df = pd.DataFrame({
    "Chartnumber" : chart_numbers,
    "Visit Count":visit_count,
    "First Service After Enrollment" : first_service_after_enrollment,
    "First Service After Intake":first_service_after_intake,
    "Days After Intake":days_after_intake,
    "Days between MH Infusion and Intake": days_btn_mhinfusion_intake,
    "Number of MH infusions": mhInfusion_count,
    "Number of MH infusions outside Cambridge":infusion_count_outside_cambridge,
    "Number of Visits outside Cambridge":visit_count_outside_cambridge
})

In [ ]:
post_intake_df

In [ ]:
stop here

In [ ]:
seen_df[seen_df["Chartnumber"]==10]["Appt Type Group"].tolist().index('MH Infusion')

In [ ]:
appointment_df[appointment_df["Facility City"]!="CAMBRIDGE"]

In [ ]:
appointment_df[appointment_df["Chartnumber"]==2194]

## Data Analysis and Visualization

# Conclusion

**Challenges**

- Not all patients in the records started with an intake. Some had no intakes in the appointment records. Also, theere are some patients that had other services before intake for instance chart#s 24 and 118
- It is cumbersome to focus this analysis on cambridge since some patients visited cambridge and other facilities for instance chart# 2194. - Fix this by counting the number of visits and infusions outside the Cambridge facility.
-